In [1]:
#Importing needed libraries
import pandas as pd
import numpy as np
import scipy
import traceback
import scanpy as sc
import anndata as ad

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay, roc_auc_score, precision_recall_curve, classification_report

#Scanorama Batch Normalization 
import scanorama

In [2]:
# Reading the existing data into dataframes 
df_dichek = pd.read_csv('all_data_matrix_logNorm.csv', index_col=0,header=0).T
df_pedroza = pd.read_csv('pedroza_data_matrix_logNorm.csv',index_col=0,header=0).T

df_dichek_metadata = pd.read_csv('all_metadata.csv',index_col=0,header=0)
df_pedroza_metadata = pd.read_csv('pedroza_metadata.csv',index_col=0,header=0)

#Changing indices so that they match and the two data sets can be joined together 
df_dichek.index = df_dichek.index.str.replace('.','-')
df_pedroza.index = df_pedroza.index.str.replace('.', '-')

#make it so that df_dichek and df_pedroza have the same genes involved 
intersecting_genes = df_dichek.columns.intersection(df_pedroza.columns)
X_dichek = df_dichek[intersecting_genes]
X_pedroza = df_pedroza[intersecting_genes]

# Split metadata into the SHF = 0, CNC = 1 
y_dichek = (df_dichek_metadata['lineage']=='CNC').astype('int')
y_pedroza  = (df_pedroza_metadata['lineage'] == 'CNC').astype('int')
print("y_dichek shape: ",y_dichek.shape)
print("y_pedroza shape: ", y_pedroza.shape)
print(y_pedroza.head(5))
print(y_dichek.head(5))

y_dichek shape:  (15431,)
y_pedroza shape:  (9745,)
pos_M_wt_AAACCCACATCTCGTC-1    0
pos_M_wt_AAACCCAGTGACACGA-1    0
pos_M_wt_AAACGAAGTATGAAGT-1    0
pos_M_wt_AAACGCTAGAGGTTTA-1    0
pos_M_wt_AAACGCTGTGGCTCTG-1    0
Name: lineage, dtype: int64
MY506pos_AAACCCATCGCCATAA-1    0
MY506pos_AAACGAAAGTGGTTGG-1    0
MY506pos_AAACGCTAGACAGTCG-1    0
MY506pos_AAAGGGCGTCGCAACC-1    0
MY506pos_AAAGTCCCAACAGTGG-1    0
Name: lineage, dtype: int64


In [3]:
# transform into np arrays 
np_dichek = X_dichek.to_numpy()
np_pedroza = X_pedroza.to_numpy()
datasets = [np_dichek, np_pedroza]

In [4]:
num_genes = intersecting_genes.size
genes_list = list()
for i in range(0, num_genes): 
    genes_list.append(intersecting_genes[i])
    
nested_list = list()
nested_list.append(genes_list.copy())
nested_list.append(genes_list.copy())

In [5]:
integrated, corrected, genes = scanorama.correct(datasets, nested_list, return_dimred=True)

Found 16855 genes among all datasets
[[0.         0.45253976]
 [0.         0.        ]]
Processing datasets (0, 1)


In [6]:
df_corrected_dichek = pd.DataFrame.sparse.from_spmatrix(corrected[0])
df_corrected_pedroza = pd.DataFrame.sparse.from_spmatrix(corrected[1])
print(df_corrected_dichek.shape)
print(df_corrected_pedroza.shape)
integrated_dichek = integrated[0]
integrated_pedroza = integrated[1]

(15431, 16855)
(9745, 16855)


# Test on Both Datasets 

In [7]:
#Combine both datasets 
X = pd.concat([df_corrected_dichek, df_corrected_pedroza])
y = pd.concat([y_dichek, y_pedroza])

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

rf = RandomForestClassifier()
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
y_prob = rf.predict_proba(X_test)

y_prob_new = np.empty((y_prob.shape[0], ))
for i in range(0, y_prob.shape[0]):
    y_prob_new[i] = y_prob[i][1]
    
# Accuracy Classification Score 
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auc_score = roc_auc_score(y_test, y_prob_new)
classification = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("ROC AUC :", auc_score)
print(classification)

Accuracy: 0.9308975377283558
Precision: 0.9267313482770158
Recall: 0.955501897205933
ROC AUC : 0.98097297843495
              precision    recall  f1-score   support

           0       0.94      0.90      0.92      2137
           1       0.93      0.96      0.94      2899

    accuracy                           0.93      5036
   macro avg       0.93      0.93      0.93      5036
weighted avg       0.93      0.93      0.93      5036



This performed fairly well, so I will try subsetting the data and see at which point it begins to behave poorly. 

In [19]:
X = pd.concat([df_corrected_dichek, df_corrected_pedroza])
y = pd.concat([y_dichek, y_pedroza])
for i in range (0, 14):
    test_set_size = 0.25 + (0.05 * i)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_set_size)
    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)

    y_pred = rf.predict(X_test)
    y_prob = rf.predict_proba(X_test)

    y_prob_new = np.empty((y_prob.shape[0], ))
    for j in range(0, y_prob.shape[0]):
        y_prob_new[j] = y_prob[j][1]

    # Accuracy Classification Score 
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    auc_score = roc_auc_score(y_test, y_prob_new)
    classification = classification_report(y_test, y_pred)
 
    print("Test size:", test_set_size)
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("ROC AUC :", auc_score)
    print(classification)

Test size: 0.25
Accuracy: 0.9240546552272005
Precision: 0.9146659707724426
Recall: 0.9586980306345733
ROC AUC : 0.9802154133411194
              precision    recall  f1-score   support

           0       0.94      0.88      0.91      2638
           1       0.91      0.96      0.94      3656

    accuracy                           0.92      6294
   macro avg       0.93      0.92      0.92      6294
weighted avg       0.92      0.92      0.92      6294

Test size: 0.3
Accuracy: 0.9314179796107507
Precision: 0.9235989492119089
Recall: 0.9614858705560619
ROC AUC : 0.980479470795693
              precision    recall  f1-score   support

           0       0.94      0.89      0.92      3165
           1       0.92      0.96      0.94      4388

    accuracy                           0.93      7553
   macro avg       0.93      0.93      0.93      7553
weighted avg       0.93      0.93      0.93      7553

Test size: 0.35
Accuracy: 0.9293009532455743
Precision: 0.9198773241326433
Recall: 0.9

In [7]:
X = pd.concat([df_corrected_dichek, df_corrected_pedroza])
y = pd.concat([y_dichek, y_pedroza])
for i in range (1, 10):
    test_set_size = 0.9 + (0.01 * i)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_set_size)
    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)

    y_pred = rf.predict(X_test)
    y_prob = rf.predict_proba(X_test)

    y_prob_new = np.empty((y_prob.shape[0], ))
    for j in range(0, y_prob.shape[0]):
        y_prob_new[j] = y_prob[j][1]

    # Accuracy Classification Score 
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    auc_score = roc_auc_score(y_test, y_prob_new)
    classification = classification_report(y_test, y_pred)
 
    print("Test size:", test_set_size)
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("ROC AUC :", auc_score)
    print(classification)

Test size: 0.91
Accuracy: 0.8881323381781677
Precision: 0.8672609513197298
Recall: 0.9498855835240274
ROC AUC : 0.963483804443747
              precision    recall  f1-score   support

           0       0.92      0.81      0.86      9801
           1       0.87      0.95      0.91     13110

    accuracy                           0.89     22911
   macro avg       0.90      0.88      0.88     22911
weighted avg       0.89      0.89      0.89     22911

Test size: 0.92
Accuracy: 0.8998791123391763
Precision: 0.8782951636338476
Recall: 0.95774860419496
ROC AUC : 0.9678959167665292
              precision    recall  f1-score   support

           0       0.94      0.82      0.88      9908
           1       0.88      0.96      0.92     13254

    accuracy                           0.90     23162
   macro avg       0.91      0.89      0.90     23162
weighted avg       0.90      0.90      0.90     23162

Test size: 0.93
Accuracy: 0.8911335098658922
Precision: 0.8640894396551724
Recall: 0.96

In [8]:
X = pd.concat([df_corrected_dichek, df_corrected_pedroza])
y = pd.concat([y_dichek, y_pedroza])
for i in range (1, 10):
    test_set_size = 0.99 + (0.001 * i)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_set_size)
    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)

    y_pred = rf.predict(X_test)
    y_prob = rf.predict_proba(X_test)

    y_prob_new = np.empty((y_prob.shape[0], ))
    for j in range(0, y_prob.shape[0]):
        y_prob_new[j] = y_prob[j][1]

    # Accuracy Classification Score 
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    auc_score = roc_auc_score(y_test, y_prob_new)
    classification = classification_report(y_test, y_pred)
 
    print("Test size:", test_set_size)
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("ROC AUC :", auc_score)
    print(classification)

Test size: 0.991
Accuracy: 0.7116633266533066
Precision: 0.6979131793326637
Recall: 0.8754199328107503
ROC AUC : 0.7942591242238608
              precision    recall  f1-score   support

           0       0.75      0.49      0.59     10662
           1       0.70      0.88      0.78     14288

    accuracy                           0.71     24950
   macro avg       0.72      0.68      0.68     24950
weighted avg       0.72      0.71      0.70     24950

Test size: 0.992
Accuracy: 0.7332132132132132
Precision: 0.7111332337813174
Recall: 0.8992866135123794
ROC AUC : 0.8210267394261221
              precision    recall  f1-score   support

           0       0.79      0.51      0.62     10677
           1       0.71      0.90      0.79     14298

    accuracy                           0.73     24975
   macro avg       0.75      0.71      0.71     24975
weighted avg       0.75      0.73      0.72     24975

Test size: 0.993
Accuracy: 0.69396
Precision: 0.6631315363087347
Recall: 0.9447397

# Train on Dichek, Test on Pedroza 

In [8]:
X_train = df_corrected_dichek 
y_train = y_dichek 
X_test = df_corrected_pedroza
y_test = y_pedroza 

rf = RandomForestClassifier()
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
y_prob = rf.predict_proba(X_test)

y_prob_new = np.empty((y_prob.shape[0], ))
for i in range(0, y_prob.shape[0]):
    y_prob_new[i] = y_prob[i][1]

# Accuracy Classification Score 
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auc_score = roc_auc_score(y_test, y_prob_new)
classification = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("ROC AUC :", auc_score)
print(classification)

Accuracy: 0.5127757824525397
Precision: 0.53785594639866
Recall: 0.6175
ROC AUC : 0.5083585935516628
              precision    recall  f1-score   support

           0       0.47      0.39      0.43      4545
           1       0.54      0.62      0.57      5200

    accuracy                           0.51      9745
   macro avg       0.51      0.51      0.50      9745
weighted avg       0.51      0.51      0.51      9745



As can be seen above, there are issues with the batch correction since the prediction accuracy is not large enough and the ROC AUC isn't that good either. To troubleshoot, Professor Wang suggested going through and adding in data points from the othe datasets mice to see if that helps at all with the accuracy and seeing at what point that the dataset reaches a higher accuracy. We use this as a metric to diagnose the issue at hand. 

In [101]:
nCells_dichek = df_corrected_dichek.shape[0]
nCells_pedroza = df_corrected_pedroza.shape[0]

for i in range (0, 20): 
    



Num Cells from Pedroza: 1000
Accuracy: 0.40537449971412237
Precision: 0.0
Recall: 0.0
ROC AUC : 0.5004681295432354
              precision    recall  f1-score   support

           0       0.41      1.00      0.58      3545
           1       0.00      0.00      0.00      5200

    accuracy                           0.41      8745
   macro avg       0.20      0.50      0.29      8745
weighted avg       0.16      0.41      0.23      8745



/local1/azregan/ENTER/envs/cse428/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/local1/azregan/ENTER/envs/cse428/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/local1/azregan/ENTER/envs/cse428/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/local1/azregan/ENT

# Train on Pedroza, Test on Dichek 

In [9]:
X_train = df_corrected_pedroza
y_train = y_pedroza
X_test = df_corrected_dichek 
y_test = y_dichek

rf = RandomForestClassifier()
rf.fit(X_train, y_train)


y_pred = rf.predict(X_test)
y_prob = rf.predict_proba(X_test)

y_prob_new = np.empty((y_prob.shape[0], ))
for i in range(0, y_prob.shape[0]):
    y_prob_new[i] = y_prob[i][1]

# Accuracy Classification Score 
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auc_score = roc_auc_score(y_test, y_prob_new)
classification = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("ROC AUC :", auc_score)
print(classification)

Accuracy: 0.539887239971486
Precision: 0.6248519308220801
Recall: 0.572871416159861
ROC AUC : 0.5474666894607643
              precision    recall  f1-score   support

           0       0.44      0.49      0.46      6223
           1       0.62      0.57      0.60      9208

    accuracy                           0.54     15431
   macro avg       0.53      0.53      0.53     15431
weighted avg       0.55      0.54      0.54     15431

